# Bitcoin derivation and transaction manual

In [23]:
from pybip44 import HDPrivateKey
import requests
import pandas as pd

## Direvation

In [17]:
master_key = HDPrivateKey.master_key_from_mnemonic(
    'trade icon company use feature fee order double inhale gift news long', 'btc')

# to switch between test and main network, you need to change 'coin' in derivation path m/44'/{coin}'/0
# for Main coin is 0
# for Test coin is 1
root_key = HDPrivateKey.from_path(master_key, "m/44'/1'/0'")

private_keys = []

for i in range(10):
    private_key = HDPrivateKey.from_path(root_key, '{change}/{index}'.format(change=0, index=i))
    private_keys.append(private_key)
    public_key = private_key.public_key
    print(f'Index {i}:')
    print(f'\tPublic Key: {public_key.to_hex()}')
    print(f'\tPrivate Key: {private_key.to_hex()}')
    
    # and here you need to change main_net parameter
    print(f'\tAddress: {public_key.address(main_net=False)}')
    print('-' * 100)

Index 0:
	Public Key: 024580d58d11e1afd37e71649e786fc5129f4ccd611a173a22a169c386d4da08b3
	Private Key: 02ef7b9791c17f35bb7f18f46209c984dc8a0317813166e5c00e7c06af5bdb9b
	Address: mtQbvrrZd9kDLLNfJHaHHhu1t3jHx1c3Jh
----------------------------------------------------------------------------------------------------
Index 1:
	Public Key: 027739a5d095625e79f393bc402ca9c1172818cacc1020bfabd6e4feb6b958842a
	Private Key: ab45ea109e4f9952f57da244af9c5b548eacdbb094fe3ea56fa5a219415201a5
	Address: mrG7g5qttebyfWar9MjYex9aDsDdVFGFLg
----------------------------------------------------------------------------------------------------
Index 2:
	Public Key: 0279bb5b80f2234d75d756539d2c011faf51973a112883c05e49fce043949232a9
	Private Key: 2a8277a4528549725dbb640a7f0d8547817b0ee40bf438575d64ca8719a13071
	Address: mwPdt1EndTm1C1CZyXNUjYmKACo5JTRonZ
----------------------------------------------------------------------------------------------------
Index 3:
	Public Key: 037968e4488f5ea53ce5857e2a3759a6985a

## Balances

In [25]:
addresses = [key.public_key.address(main_net=False) for key in private_keys]
decimals = 10 ** 8

# here test net node on 5000 port
# and main net on 5001 port
url = 'http://78.155.199.25:5000'
auth_token='5a18fa63abf647ddb6483c9e241a4765'
hed = {'Authorization': 'Bearer ' + auth_token}

balances = []

for address in addresses:
    response = requests.get(f'{url}/{address}/balance', headers=hed)
    if response:
        balances.append(response.json()['confirmed'] / decimals)

df = pd.DataFrame(zip(addresses,balances), columns = ['Address', 'Balance (confirmed)']) 

df

,Address,Balance (confirmed)
0,mtQbvrrZd9kDLLNfJHaHHhu1t3jHx1c3Jh,0.025732
1,mrG7g5qttebyfWar9MjYex9aDsDdVFGFLg,0.026889
2,mwPdt1EndTm1C1CZyXNUjYmKACo5JTRonZ,0.008749
3,mssTb1GWSotwnq4UVxhKashVhZLo1zLqEC,0.002685
4,mr6fvKLjX2vsctMgirAMbSrYorPijUbpeK,0.000430
5,mn6ooSwPCwR4NpW6Ce8TKvy85zVM8tcY7j,0.000958
6,mnXM94JpsQjawV7b4m6WuZRj9dBijhamBg,0.000381
7,msFoXfhbwMKsVzNsj39hZeVhrmWBQJQ5iK,0.000671
8,mxVzrH4MSieXT4KbG53cQ4BmFw66kQuKnG,0.001601
9,mmSzVX7xnRBNTDq9ahJzZZCL3zfanUizaU,0.000048
